## DIA 048: Implementación de un Sistema de Análisis de Tendencias Basado en Feedback

In Day 48, we add a new administrative endpoint that analyzes the historical feedback collected from users. This system aggregates key metrics, including the total number of feedback entries, the percentage of correct versus incorrect predictions, and a distribution of prediction values. The goal is to offer administrators actionable insights into the model’s performance over time, helping to identify trends and areas for improvement. Access to this endpoint is protected by JWT and role-based authorization (only admins can access it).

Complete Code (api.py)
python
Copiar
import os
import io
import random
import json
import time
import threading
import logging
import hashlib
from datetime import datetime
from functools import wraps

import requests
import redis
import numpy as np
from PIL import Image

from flask import Flask, request, jsonify, render_template, url_for
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from flask_sqlalchemy import SQLAlchemy
from flask_migrate import Migrate
from flask_bcrypt import Bcrypt
from flask_mail import Mail, Message
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address
from flask_socketio import SocketIO, emit, join_room
from flasgger import Swagger

# Basic configuration and environment variables
app = Flask(__name__)
app.config['SECRET_KEY'] = os.getenv('SECRET_KEY', 'your_secret_key')
app.config['JWT_SECRET_KEY'] = os.getenv('JWT_SECRET_KEY', 'your_jwt_secret_key')
app.config['SQLALCHEMY_DATABASE_URI'] = os.getenv('DATABASE_URL', 'sqlite:///app.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

# Flasgger configuration for automatic documentation
swagger_config = {
    "headers": [],
    "specs": [
        {
            "endpoint": "apispec_1",
            "route": "/apispec_1.json",
            "rule_filter": lambda rule: True,
            "model_filter": lambda tag: True,
        }
    ],
    "static_url_path": "/flasgger_static",
    "swagger_ui": True,
    "specs_route": "/docs/"
}
swagger = Swagger(app, config=swagger_config)

# Initialize extensions
db = SQLAlchemy(app)
migrate = Migrate(app, db)
bcrypt = Bcrypt(app)
mail = Mail(app)
jwt = JWTManager(app)
limiter = Limiter(app, key_func=get_remote_address, default_limits=["200 per day", "50 per hour"])
socketio = SocketIO(app, cors_allowed_origins="*")

# Configure Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Redis
redis_client = redis.Redis(
    host=os.getenv("REDIS_HOST", "localhost"),
    port=int(os.getenv("REDIS_PORT", 6379)),
    db=0
)

# ---------------------------
# Models
# ---------------------------
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    # Additional fields (e.g., email, role) can be added as needed

class Feedback(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    prediction = db.Column(db.Integer, nullable=False)
    correct = db.Column(db.Boolean, nullable=False)
    comment = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "prediction": self.prediction,
            "correct": self.correct,
            "comment": self.comment,
            "timestamp": self.timestamp.isoformat()
        }

# New Model: AuditLog for auditing
class AuditLog(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), nullable=False)
    action = db.Column(db.String(120), nullable=False)
    details = db.Column(db.Text, nullable=True)
    timestamp = db.Column(db.DateTime, default=datetime.utcnow)

    def to_dict(self):
        return {
            "id": self.id,
            "username": self.username,
            "action": self.action,
            "details": self.details,
            "timestamp": self.timestamp.isoformat()
        }

# ---------------------------
# Function to Log Audit Events
# ---------------------------
def log_audit_event(username, action, details=""):
    audit = AuditLog(username=username, action=action, details=details)
    db.session.add(audit)
    db.session.commit()
    logger.info(f"Audit log: {username} - {action} - {details}")

# ---------------------------
# Role Decorator (simplified)
# ---------------------------
def role_required(required_role):
    def decorator(f):
        @wraps(f)
        def wrapper(*args, **kwargs):
            current_user_identity = get_jwt_identity()
            if not current_user_identity:
                return jsonify({"msg": "Token de acceso requerido"}), 401
            user = User.query.filter_by(username=current_user_identity).first()
            if not user:
                return jsonify({"msg": "Usuario no encontrado"}), 404
            if getattr(user, 'role', 'user') != required_role:
                return jsonify({"msg": "Acceso no autorizado"}), 403
            return f(*args, **kwargs)
        return wrapper
    return decorator

# ---------------------------
# Common Endpoints
# ---------------------------
@app.route('/login', methods=['POST'])
def login():
    """
    User Login
    ---
    tags:
      - Auth
    parameters:
      - in: body
        name: credentials
        schema:
          type: object
          required:
            - username
            - password
          properties:
            username:
              type: string
            password:
              type: string
    responses:
      200:
        description: JWT token generated.
    """
    data = request.get_json()
    username = data.get('username')
    password = data.get('password')
    if not username or not password:
        return jsonify({"msg": "Username and password required"}), 400
    token = create_access_token(identity=username)
    log_audit_event(username, "login", "Usuario inició sesión")
    logger.info(f"Usuario '{username}' inició sesión.")
    return jsonify(access_token=token), 200

@app.route('/health', methods=['GET'])
def health():
    """
    Health Check
    ---
    tags:
      - Health
    responses:
      200:
        description: The application is running.
    """
    return jsonify({"status": "ok"}), 200

# ---------------------------
# Endpoint: Prediction with Advanced Caching (v1)
# ---------------------------
api_v1 = Blueprint('api_v1', __name__)
@api_v1.route('/predict', methods=['POST'])
@jwt_required()
@limiter.limit("100 per day")
def predict_v1():
    """
    Predict Endpoint - v1 with Caching
    ---
    tags:
      - Prediction v1
    consumes:
      - multipart/form-data
    parameters:
      - in: formData
        name: file
        type: file
        required: true
        description: Image file to predict the digit.
    responses:
      200:
        description: Prediction result.
        schema:
          type: object
          properties:
            prediccion:
              type: integer
            probabilidad:
              type: number
            version:
              type: string
            cached:
              type: boolean
    """
    if 'file' not in request.files:
        return jsonify({"error": "No se encontró el archivo"}), 400
    file = request.files['file']
    if file.filename == '':
        return jsonify({"error": "No se seleccionó ningún archivo"}), 400

    file_bytes = file.read()
    cache_key = hashlib.md5(file_bytes).hexdigest()
    cached_result = redis_client.get(cache_key)
    if cached_result:
        logger.info("Resultado obtenido de la caché para la clave: %s", cache_key)
        result = json.loads(cached_result)
        result['cached'] = True
        log_audit_event(get_jwt_identity(), "predict_v1", "Resultado obtenido de la caché")
        return jsonify(result), 200

    image = Image.open(io.BytesIO(file_bytes)).convert('L')
    image = image.resize((28, 28))
    image_array = np.array(image).astype('float32') / 255.0
    image_array = np.expand_dims(image_array, axis=0)
    image_array = np.expand_dims(image_array, axis=-1)
    result = {"prediccion": 5, "probabilidad": 0.90, "version": "v1", "cached": False}
    redis_client.setex(cache_key, 600, json.dumps(result))
    logger.info("v1: Predicción realizada y almacenada en caché con clave: %s", cache_key)
    log_audit_event(get_jwt_identity(), "predict_v1", "Predicción procesada y cacheada")
    return jsonify(result), 200

app.register_blueprint(api_v1, url_prefix='/api/v1')

# ---------------------------
# New Endpoint: Trends Analysis from Feedback
# ---------------------------
@app.route('/admin/trends', methods=['GET'])
@jwt_required()
@role_required('admin')
def analyze_trends():
    """
    Analyze Trends from Feedback
    ---
    tags:
      - Trends
    responses:
      200:
        description: Feedback trends analyzed.
        schema:
          type: object
          properties:
            total_feedback:
              type: integer
            correct_feedback:
              type: integer
            incorrect_feedback:
              type: integer
            accuracy_percentage:
              type: number
            prediction_distribution:
              type: object
    """
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200

    correct_count = Feedback.query.filter_by(correct=True).count()
    incorrect_count = Feedback.query.filter_by(correct=False).count()
    accuracy_percentage = (correct_count / total_feedback) * 100

    distribution = {}
    predictions = Feedback.query.with_entities(Feedback.prediction, db.func.count(Feedback.prediction)).group_by(Feedback.prediction).all()
    for prediction, count in predictions:
        distribution[str(prediction)] = count

    trends = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_count,
        "incorrect_feedback": incorrect_count,
        "accuracy_percentage": accuracy_percentage,
        "prediction_distribution": distribution
    }
    log_audit_event(get_jwt_identity(), "analyze_trends", "Análisis de tendencias de feedback realizado")
    logger.info("Tendencias analizadas: %s", trends)
    return jsonify(trends), 200

# ---------------------------
# New Endpoint: Audit Logs (for Admins)
# ---------------------------
@app.route('/admin/audit_logs', methods=['GET'])
@jwt_required()
@role_required('admin')
def get_audit_logs():
    """
    Get Audit Logs
    ---
    tags:
      - Audit
    responses:
      200:
        description: List of audit log entries.
    """
    logs = AuditLog.query.order_by(AuditLog.timestamp.desc()).all()
    log_list = [log.to_dict() for log in logs]
    return jsonify(log_list), 200

# ---------------------------
# New Endpoint: Slack Alert (for Admins)
# ---------------------------
def send_slack_notification(message):
    webhook_url = os.getenv("SLACK_WEBHOOK_URL")
    if not webhook_url:
        logger.error("SLACK_WEBHOOK_URL no está configurado.")
        return False
    payload = {"text": message}
    headers = {"Content-Type": "application/json"}
    response = requests.post(webhook_url, data=json.dumps(payload), headers=headers)
    if response.status_code != 200:
        logger.error("Error al enviar notificación a Slack: %s", response.text)
        return False
    return True

@app.route('/admin/slack_alert', methods=['POST'])
@jwt_required()
@role_required('admin')
def slack_alert():
    """
    Slack Alert
    ---
    tags:
      - Alerts
    parameters:
      - in: body
        name: alert
        schema:
          type: object
          required:
            - message
          properties:
            message:
              type: string
    responses:
      200:
        description: Slack alert sent successfully.
      500:
        description: Failed to send Slack alert.
    """
    data = request.get_json()
    message = data.get("message", "Alerta de la API")
    if send_slack_notification(message):
        logger.info("Notificación Slack enviada: " + message)
        log_audit_event(get_jwt_identity(), "slack_alert", message)
        return jsonify({"msg": "Slack alert sent successfully"}), 200
    else:
        return jsonify({"msg": "Failed to send Slack alert"}), 500

# ---------------------------
# New Endpoint: Retraining with MLFlow (for Admins)
# ---------------------------
@app.route('/admin/retrain_mlflow', methods=['POST'])
@jwt_required()
@role_required('admin')
def retrain_mlflow():
    mlflow.set_experiment("Retraining Experiment")
    with mlflow.start_run() as run:
        logger.info("Inicio del retraining con MLFlow...")
        time.sleep(10)  # Simula retraining
        mlflow.log_param("learning_rate", 0.001)
        mlflow.log_metric("accuracy", 0.92)
        artifact_path = "model_info.txt"
        with open(artifact_path, "w") as f:
            f.write("Modelo actualizado basado en feedback con MLFlow.")
        mlflow.log_artifact(artifact_path)
        logger.info("Retraining completado y registrado en MLFlow.")
        log_audit_event(get_jwt_identity(), "retrain_mlflow", f"Run ID: {run.info.run_id}")
        return jsonify({"msg": "Retraining experiment logged in MLFlow", "run_id": run.info.run_id}), 202

# ---------------------------
# New Endpoint: Notification Email for Critical Errors (for Admins)
# ---------------------------
def send_error_email(subject, body):
    admin_email = os.getenv("ADMIN_EMAIL", "admin@example.com")
    msg = Message(subject=subject, recipients=[admin_email], body=body)
    try:
        mail.send(msg)
        logger.info("Correo de alerta enviado a %s", admin_email)
    except Exception as e:
        logger.error("Error enviando correo de alerta: %s", str(e))

@app.errorhandler(500)
def internal_error(error):
    error_message = f"Error Crítico: {str(error)}"
    send_error_email("Error Crítico en la API", error_message)
    logger.exception("Error interno: %s", error)
    return jsonify({"error": "Internal Server Error"}), 500

# ---------------------------
# New Endpoint: Trends Analysis (for Admins)
# ---------------------------
@app.route('/admin/trends', methods=['GET'])
@jwt_required()
@role_required('admin')
def analyze_trends():
    total_feedback = Feedback.query.count()
    if total_feedback == 0:
        return jsonify({"msg": "No hay feedback disponible"}), 200

    correct_count = Feedback.query.filter_by(correct=True).count()
    incorrect_count = Feedback.query.filter_by(correct=False).count()
    accuracy_percentage = (correct_count / total_feedback) * 100

    distribution = {}
    predictions = Feedback.query.with_entities(Feedback.prediction, db.func.count(Feedback.prediction)).group_by(Feedback.prediction).all()
    for prediction, count in predictions:
        distribution[str(prediction)] = count

    trends = {
        "total_feedback": total_feedback,
        "correct_feedback": correct_count,
        "incorrect_feedback": incorrect_count,
        "accuracy_percentage": accuracy_percentage,
        "prediction_distribution": distribution
    }
    log_audit_event(get_jwt_identity(), "analyze_trends", "Análisis de tendencias de feedback realizado")
    logger.info("Tendencias analizadas: %s", trends)
    return jsonify(trends), 200

# ---------------------------
# Execute the application with WebSocket and Swagger support
# ---------------------------
if __name__ == '__main__':
    socketio.run(app, debug=True)
Explanation
Redis Caching in /api/v1/predict:
The endpoint computes a unique cache key using MD5 on the image bytes. It checks if a cached result exists in Redis and returns it if available. If not, it processes the image (simulated prediction), caches the result for 10 minutes, and logs the operation.

Audit Logging:
The AuditLog model stores critical events. The helper function log_audit_event is used throughout the endpoints (login, predictions, trends, etc.) to record actions for traceability.

Critical Error Email Alerts:
A custom error handler for HTTP 500 errors sends an email to the administrator (configured via ADMIN_EMAIL) using Flask-Mail. This alerts the admin of any critical internal errors.

Additional Admin Endpoints:
Several admin endpoints are implemented:

/admin/trends analyzes feedback data and returns aggregated metrics.
/admin/slack_alert sends notifications to Slack using a webhook.
/admin/retrain_mlflow simulates a retraining process and logs the experiment in MLFlow.
/admin/audit_logs (if needed) could provide access to audit logs (not shown here but available via the AuditLog model).
Security:
Endpoints that modify or access sensitive data (e.g., admin endpoints) are protected with JWT and a role-based decorator (role_required).

Execution:
The app runs using socketio.run(app, debug=True) to support both HTTP and WebSocket connections, and documentation is available via Flasgger at /docs/.